# Behavioral Exploration:
# Topic Modeling and SNR
 
Exploring the research question: what effect does modifying the input corpus have on the ability to do topic modeling?

TASKS:

- Build the system.
- Test the system under various constraints & make observations about its behaviour.

## Sections

The notebook first establishes a fixed topic model (generic LDA). Then, it defines some properties of corpora that may be modified during experiments. It loads a few corpora and examines their properties. The generic topic model is used to extract topics from these corpora. Performance metrics for the topic model that may be affected by changes in the corpora are defined and calculated on the corpora.

- [Section 1: A Fixed Approach](#1-A-Fixed-Approach)
- [Section 2: Corporus Characteristics](#2-Corpus-Characteristics)
- [Section 3: Choosing Corpora](#3-Choosing-Corpora)
  - [Section 3.1: Wine Reviews](#3.1-Wine-Reviews)
  - [Section 3.2: Brown](#3.2-Brown)
  - [Section 3.3: ABC](#3.3-ABC)
  - [Section 3.4: Genesis](#3.3-Genesis)
  - [Section 3.5: Inaugural](#3.3-Inaugural)
  - [Section 3.6: State of the Union](#3.3-State-of-the-Union)
- [Section 4: Performance Metrics](#4-Performance-Metrics)
- [Section 5: Exploring Results](#5-Exploring-Results)

In [1]:
%load_ext autoreload
%autoreload 2

# 1 A Fixed Approach

This section constructs a baseline model for topic-modeling that is up-to-date on the latest settings for optimal performance. The most widely used and successful topic-modeling approach is [Latent Dirichlet Allocation]() [Blei].

It performs best under the following settings:
- Asymmetric Alpha
- Symmetric Beta

The appropriate number of topics is still somewhat up in the air. Beli et al. say K should be less than the number of documents. TBD is an upper bound on K.

An interesting experiment would be to see on which K the Brown corpus converges since this is theorized to be a relatively broad sample of the English language. It begs the research question, How many topics are there in the Human language?

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

K = 100
ALPHA = 1/K
BETA = 1/K

# TODO: MAKE ALPHA ASYMMETRIC
lda = LatentDirichletAllocation(n_components=K,
                                doc_topic_prior=ALPHA,
                                topic_word_prior=BETA,
                                learning_method='online')

# This is reusable, we'll fit it separately to each corpus
# Converts a list of documents into their word-counts,
# after removing stopwords.
tf_vectorizer = CountVectorizer(stop_words='english')

def get_model(data_samples):
    # Build the model
    model_components = {}
    tf = tf_vectorizer.fit_transform(data_samples)
    lda.fit(tf) 
    # Piece together model components
    model_components['features'] = tf_vectorizer.get_feature_names()
    model_components['topic_word'] = lda.components_
    model_components['doc_word'] = tf
    model_components['doc_topic'] = lda.transform(tf)
    return model_components

def get_top_words(model, feature_names, n_top_words=10):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        sorted_top = topic.argsort()[:-n_top_words-1:-1]
        top_words[topic_idx] = [feature_names[i] for i in sorted_top]
    return top_words

def print_top_words(model, feature_names, n_top_words=10):
    top_words = get_top_words(lda,feature_names,n_top_words)
    for topic in top_words:
        words = ' '.join(top_words[topic])
        print("Topic_{}: {}".format(topic,words))

# 2 Corpus Characteristics

I decided it would be easier to define an new corpus reader object that extends NLTK's existing PlaintextCorpusReader. This can be found in `corpus.py`. It extends PlaintextCorpusReader by adding characteristics to the corpus so that the user can ask basic questions like how many documents are there? What is their average word count? A lot of the characteristics are straightforward. For those that aren't, an explanation is provided.

So far the characteristics I have defined are:
- Number of documents
- Average document length (in words)
- Vocab size
- Readability ([smog index](https://en.wikipedia.org/wiki/SMOG))
- Distance from uniform distribution
- [Lexical diversity](http://textinspector.com/help/?page_id=136)
- Stopword presence (what percentage of the corpus is stopwords?)

Still TODO are:
- Stability
- Inherent Topics

# 3 Choosing Corpora

In this section I'm going to load some various real-world corpora and test out the different metrics. This is really the __behavioural exploration__ section. After fixing any bugs in the metrics, I'll start making some initial observations of the system to see if I can find hints of phenomena or correlations.

In [3]:
# The corpus reader I created to make things easier
from corpus import PropertiesCorpusReader

## 3.1 Wine Reviews

This first corpus is a collection of wine reviews. It's an example of a small corpus with short-text documents. Unfortunately they've all been joined into one big file so we need to do a little manual labour to separate the individual short reviews. This means creating a new nltk corpus reader object from the one big wine review document.

In [31]:
# Skip this cell unless the wine files are missing (in corpus folder)
import os

from nltk.corpus import webtext
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

wine_doc = webtext.raw('wine.txt')

# Individual reviews are split by newline characters
wine_reviews = list(filter(None,wine_doc.split('\n')))

if not os.path.exists('corpus/wine'):
    os.makedirs('corpus/wine')
    
fileids = []
for i in range(len(wine_reviews)):
    fn = '{:04}.txt'.format(i)
    fileids.append(fn)
    with open(os.path.join('wine_files',fn),'w') as f:
        f.write(wine_reviews[i])
    f.close()

Load the wine files as an NLTK corpus reader object.

In [ ]:
wine_cr = PlaintextCorpusReader('corpus/wine_files',
                                fileids,
                                word_tokenizer=webtext._word_tokenizer,
                                sent_tokenizer=webtext._sent_tokenizer,
                                para_block_reader=webtext._para_block_reader,
                                encoding=webtext._encoding)

Now let's convert the new corpus reader into a PropertiesCorpusReader object and explore its characteristics. 

In [5]:
wine_pcr = PropertiesCorpusReader(wine_cr)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [6]:
wine_pcr.print_properties()

Number of documents: 1230
Average document length: 25.4943
Vocab size: 3417
Readability: 276.1483
Distance from uniform: 0.8687
Lexical diversity: 0.1090
Stopword presence: 0.2613


After gathering characteristics of the corpus, the next step is to see how well the generic LDA model can decipher topics from it.

In [7]:
data_samples = wine_pcr.raw_docs()
wine_components = get_model(data_samples)

The following are the matrices we need to calculate model performance. They are:

|Name|Shape|Description|
|:-|:-|:-|:-|
|topic_word|(n_topics,n_words)|Distribution of vocabulary words in each topic. Each cell answers the question: What is the likelihood of seeing this word in this topic?|
|document_word|(n_documents,n_words)|Count of vocabulary words in each document. Each cell answers the question: How many times did this word appear in this document? __This is a SPARSE MATRIX__|
|document_topic|(n_documents,n_topics)|The proportion of each document that is made up by the different topics. It answers the question: What percentage of this document talks about this topic?|
|features|(n_words,)|The literal words associated with each index in the topic_word and doc_word matrices.|

In [8]:
wine_components['topic_word'].shape

(100, 2589)

In [9]:
wine_components['doc_word'].shape

(1230, 2589)

In [10]:
wine_components['doc_topic'].shape

(1230, 100)

In [11]:
len(wine_components['features'])

2589

## 3.2 Brown

The Brown corpora is probably one of the most famous ones out there. It's the polar opposite of the wine reviews because instead of a few short documents, it has a lot of long documents. Which means it takes a really long time to build a PropertiesCorpusRedaer object of it, so verbosity is a nice sanity check while it's running.

In [15]:
from nltk.corpus import brown
brown_pcr = PropertiesCorpusReader(brown)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [16]:
brown_pcr.print_properties()

Number of documents: 500
Average document length: 2322.3840
Vocab size: 66939
Readability: 11.3273
Distance from uniform: 0.9709
Lexical diversity: 0.0576
Stopword presence: 0.0000


In [17]:
data_samples = brown_pcr.raw_docs()
brown_components = get_model(data_samples)

## 3.3 ABC

A corpus of rural and science news files.

We're going to keep them separate and treat them as two separate corpora.

In [18]:
import os

from nltk.corpus import abc
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

abc_rural = abc.raw('rural.txt')
abc_science = abc.raw('science.txt')

# Individual articles are split by newline characters
abc_rural_articles = list(filter(None,abc_rural.split('\n\n')))
abc_science_articles = list(filter(None,abc_science.split('\r\n\r\n')))

if not os.path.exists('abc_rural_files'):
    os.makedirs('abc_rural_files')
if not os.path.exists('abc_science_files'):
    os.makedirs('abc_science_files')
    
fileids = []
for i in range(len(abc_rural_articles)):
    fn = '{:04}.txt'.format(i)
    fileids.append(fn)
    with open(os.path.join('abc_rural_files',fn),'w') as f:
        f.write(abc_rural_articles[i])
    f.close()
abc_rural_cr = PlaintextCorpusReader('abc_rural_files',fileids)
abc_rural_pcr = PropertiesCorpusReader(abc_rural_cr)

fileids = []
for i in range(len(abc_science_articles)):
    fn = '{:04}.txt'.format(i)
    fileids.append(fn)
    with open(os.path.join('abc_science_files',fn),'w') as f:
        f.write(abc_science_articles[i])
    f.close()
abc_science_cr = PlaintextCorpusReader('abc_science_files',fileids)
abc_science_pcr = PropertiesCorpusReader(abc_science_cr)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.
Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [19]:
abc_rural_pcr.print_properties()

Number of documents: 2424
Average document length: 142.5660
Vocab size: 17222
Readability: 11.9941
Distance from uniform: 0.9205
Lexical diversity: 0.0498
Stopword presence: 0.3507


In [20]:
data_samples = abc_rural_pcr.raw_docs()
abc_rural_components = get_model(data_samples)

In [21]:
abc_science_pcr.print_properties()
data_samples = abc_science_pcr.raw_docs()
abc_science_components = get_model(data_samples)

Number of documents: 764
Average document length: 551.3495
Vocab size: 24306
Readability: 13.3439
Distance from uniform: 0.9375
Lexical diversity: 0.0577
Stopword presence: 0.3493


## 3.4 Genesis

NLTK King James Bible?

In [22]:
from nltk.corpus import genesis
genesis_pcr = PropertiesCorpusReader(genesis)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [23]:
genesis_pcr.print_properties()

Number of documents: 8
Average document length: 39408.5000
Vocab size: 25841
Readability: 8.4613
Distance from uniform: 0.9343
Lexical diversity: 0.0820
Stopword presence: 0.3755


In [24]:
data_samples = genesis_pcr.raw_docs()
genesis_components = get_model(data_samples)

## 3.5 Inaugural Address

A collection of 55 inaugural addresses from presidents.

In [25]:
from nltk.corpus import inaugural
inaugural_pcr = PropertiesCorpusReader(inaugural)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [26]:
inaugural_pcr.print_properties()

Number of documents: 56
Average document length: 2602.4107
Vocab size: 9754
Readability: 23.4806
Distance from uniform: 0.9143
Lexical diversity: 0.0669
Stopword presence: 0.4499


In [27]:
data_samples = inaugural_pcr.raw_docs()
inaugural_components = get_model(data_samples)

## 3.6 State of the Union

Collection of State of the Union addresses.

In [28]:
from nltk.corpus import state_union
state_union_pcr = PropertiesCorpusReader(state_union)

Calculating properties...
	Readability calculated.
	Distance from uniform calculated.
	Lexical diversity calculated.
	Stopword presence calculated.


In [29]:
state_union_pcr.print_properties()

Number of documents: 65
Average document length: 6151.1077
Vocab size: 14591
Readability: 17.6455
Distance from uniform: 0.9230
Lexical diversity: 0.0365
Stopword presence: 0.3888


In [32]:
data_samples = state_union_pcr.raw_docs()
state_union_components = get_model(data_samples)

# 4 Performance Metrics

Metrics should adhere to the following axioms:
- Similar models or topics should have similar scores.
- Different models or topics should have differing scores.
- Scores should change when the model changes.

We should also consider metrics that have been tested in previous work to rule them in/out as informative of performance. 

In addition, metrics should be able to be calculated on any matrices regardless of how they were created (using LDA or not).

So far, the metrics defined are:
- Exclusivity
- [Jensen-Shannon Divergence](https://stackoverflow.com/questions/15880133/jensen-shannon-divergence
- Effective size
- Average word length
- Rank1

Still TODO are:
- Classification
- Point-wise mutual information
- Coherence
- SNR

When possible, metrics are calculated separately for each individual topic.

PMI source:

[Newman et al. 2011](http://papers.nips.cc/paper/4291-improving-topic-coherence-with-regularized-topic-models.pdf)

In [33]:
import metrics

all_components = {'wine':wine_components,
                  'brown':brown_components,
                  'abc_rural':abc_rural_components,
                  'abc_science':abc_science_components,
                  'genesis':genesis_components,
                  'inaugural':inaugural_components,
                  'state_union':state_union_components}

pcrs = {'wine':wine_pcr,
        'brown':brown_pcr,
        'abc_rural':abc_rural_pcr,
        'abc_science':abc_science_pcr,
        'genesis':genesis_pcr,
        'inaugural':inaugural_pcr,
        'state_union':state_union_pcr}

properties = ['num_docs','avg_doc_len','vocab_size',
              'readability','lexical_diversity',
              'stopword_presence']

topic_metrics = ['average_word_length','exclusivity',
                 'rank1','distance_from_uniform',
                 'distance_from_corpus','effective_size',
                 'top_words']

names = ['wine','brown','abc_rural','abc_science','genesis','inaugural','state_union']


def get_metric(name,met):
    results = []
    metric_func = getattr(metrics,met)
    for i in range(K):
        results.append(metric_func(all_components[name],i))
    return results

def get_property(name,prop):
    pcr = pcrs[name]
    return getattr(pcr,prop)

Let's look at the various corpora and their properties.

In [34]:
import pandas as pd

cp_df = pd.DataFrame(columns=properties,index=names)
for index, row in cp_df.iterrows():
    for p in properties:
        row[p] = get_property(index,p)
        
cp_df

,num_docs,avg_doc_len,vocab_size,readability,lexical_diversity,stopword_presence
wine,1230,25.4943,3417,276.148,0.108967,0.261305
brown,500,2322.38,66939,11.3273,0.0576468,0
abc_rural,2424,142.566,17222,11.9941,0.0498351,0.350654
abc_science,764,551.349,24306,13.3439,0.0577023,0.349348
genesis,8,39408.5,25841,8.46133,0.0819652,0.375455
inaugural,56,2602.41,9754,23.4806,0.0669297,0.449878
state_union,65,6151.11,14591,17.6455,0.0364937,0.388803


# 5 Exploring Results

Since LDA is a BOW model, the words themselves are not important for most measures (except complexity).

For complexity, we know what affects the complexity score. If we modify that, how does LDA performance change?

TODO: Find/generate corpora that satisfy the characteristics defined above and train the baseline model on them.

In [35]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

Calculate all the x and y arrays of properties and metrics for plotting.

If you add a new metric and want to plot it, you need to run this cell first so there is something to be plotted by the next cell. Or! Skip this cell and run the next sell to load the plotting data directly from the pickle file.

In [36]:
plotting_data = {}
for n in names:
    print(n)
    print("\tCalculating corpus properties...")
    for p in properties:
        val = get_property(n,p)
        x = [val]*K
        plotting_data['{}_{}'.format(n,p)] = x
    print("\tCalculating topic metrics...")
    for tm in topic_metrics:
        y = get_metric(n,tm)
        plotting_data['{}_{}'.format(n,tm)] = y

wine
	Calculating corpus properties...
	Calculating topic metrics...
brown
	Calculating corpus properties...
	Calculating topic metrics...
abc_rural
	Calculating corpus properties...
	Calculating topic metrics...
abc_science
	Calculating corpus properties...
	Calculating topic metrics...
genesis
	Calculating corpus properties...
	Calculating topic metrics...
inaugural
	Calculating corpus properties...
	Calculating topic metrics...
state_union
	Calculating corpus properties...
	Calculating topic metrics...


In [39]:
import util
util.dump_pickle(plotting_data,"plotting_data_behaviour_exploration.pickle")

True

Alternatively...load the plotting data from a pickle.

In [40]:
import util
plotting_data = util.load_pickle("plotting_data_behaviour_exploration.pickle")

Build the plot. Tabs will be used to switch between corpus properties.

In [41]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import HoverTool, Legend, ColumnDataSource

def plot_metric(metric_name):
    colors = {'wine':'red',
              'brown':'orange',
              'abc_rural':'yellow',
              'abc_science':'green',
              'genesis':'blue',
              'inaugural':'purple',
              'state_union':'pink'}

    hover = HoverTool(tooltips=[('x','@x'),('y','@y'),('top words','@top_three')])
    
    property_tabs= []
    figs = {}
    for p in properties:
        fig = figure(x_axis_label=p,
                     y_axis_label=metric_name,
                     height=600,
                     width=800,
                     toolbar_location='above')
        
        # Make the labels legible when plots are downloaded
        fig.xaxis.axis_label_text_font_size = "30pt"
        fig.yaxis.axis_label_text_font_size = "30pt"
        fig.xaxis.major_label_text_font_size = "15pt"
        fig.yaxis.major_label_text_font_size = "15pt"
        fig.add_tools(hover)
        legend_items = []
        
        # One scatter plot for each corpus
        for n in names:
            circle = fig.circle(x='x',
                                y='y',
                                source=ColumnDataSource({'x':plotting_data['{}_{}'.format(n,p)],
                                                         'y':plotting_data['{}_{}'.format(n,metric_name)],
                                                         'top_three':[' '.join(w[:3]) for w in plotting_data['{}_top_words'.format(n)]]}),
                                size=20, color=colors[n], alpha=0.2)
            legend_items.append((n,[circle]))
                                
        # Build and add the interactive plot legend, to the right of the plot
        legend = Legend(items=legend_items,
                        click_policy='hide',
                        label_text_font_size='20pt',
                        location=(10,-30),
                        glyph_height=50,
                        glyph_width=50,
                        border_line_alpha=0)
        fig.add_layout(legend,'right')
        figs[p] = fig
        property_tabs.append(Panel(child=fig, title=p))

    tabs = Tabs(tabs=property_tabs)

    show(tabs,notebook_handle=True)

In [42]:
plot_metric('exclusivity')

In [43]:
plot_metric('rank1')

### Obervations:

The __number of documents__ seems to lead to an increase in rank1 of topics 

Increase in __lexical diversity__ of the corpus seems to cause exponential increase in the rank1 metric for topics.

Future results would probably benefit from combining average document length with the number of documents. Maybe (avg_doc_len) x (num_docs).

In [44]:
plot_metric('distance_from_uniform')

Topic distance from uniform distribution seems to be negatively impacted by both __average document length__ and __vocabulary size__. Interestingly, lexical diversity does not seem to have an impact.

As vocabulary size increases, topic distance from the uniform distribution decreases.

In [45]:
plot_metric('average_word_length')

There doesn't seem to be any correlations between the various corpus properties and their resulting topic metrics. 

In [46]:
plot_metric('distance_from_corpus')

### Observations:

As __vocab_size__ increases, the distance of topics from the uniform distribution also seems to increase. 

In [47]:
plot_metric('effective_size')

### Observations:

__Number of documents__ seems to have a positive impact on the resulting __effective topic sizes__. Genesis, inaugural, and state_union corpora all have very small number of documents and resulting near-zero effective topic size.

